In [53]:
import os

os.environ["CONFIG_APP_DIR"] = "/app/config"


In [54]:
import sys
import os
import pickle
os.chdir("/app")
from omegaconf import OmegaConf

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

In [55]:
from etl_pipeline.config import pipeline_config
cn = pipeline_config.cn

In [56]:
# alert = load_alert()

In [57]:
# payload = load_alert()


# Definition

In [58]:
from pipelines.ms.ms_pipeline import MSPipeline
from etl_pipeline.config import pipeline_config


In [59]:
with open(f'notebooks/sample/wm_address_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [60]:
payload = payload_json

In [61]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [62]:
pipeline_config.reload_pipeline_config()
_ = load_agent_configs()

2022-04-13 14:14:53,785 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml
2022-04-13 14:14:53,793 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml
2022-04-13 14:14:53,915 — main — WARNING — load_agent_config:68 — Field in agent configuration for: AP_PARTY_TYPE is not registered field in pipeline.yaml


In [63]:
engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

2022-04-13 14:14:54,025 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml
2022-04-13 14:14:54,034 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml
2022-04-13 14:14:54,149 — main — WARNING — load_agent_config:68 — Field in agent configuration for: AP_PARTY_TYPE is not registered field in pipeline.yaml


## transform standardized to cleansed

In [64]:
payload = pipeline.transform_standardized_to_cleansed(payload)

ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [65]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

2022-04-13 14:14:56,670 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_country. Ignore for HIT TYPE AGENT
2022-04-13 14:14:56,671 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_job. Ignore for HIT TYPE AGENT
2022-04-13 14:14:56,671 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_other. Ignore for HIT TYPE AGENT
2022-04-13 14:14:56,672 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_country. Ignore for HIT TYPE AGENT
2022-04-13 14:14:56,672 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_job. Ignore for HIT TYPE AGENT
2022-04-13 14:14:56,673 — main.engine — WARNING — merge_to_target_col_from_source_col

### payload

In [66]:
len(new_payloads)

2

In [67]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload['AP_TRIGGERS'].values[0]

{'Doe': {'ADDRESS1_LINE2': ['Jane Doe AA BBB'],
  'CONCAT_ADDRESS': ['Joe Black & Jane Doe 111 A 11TH ST AAA 1A New Jersey NJ']}}

In [68]:
out_payload[[i for i in out_payload.columns if "names" in i and "agg" in i]].loc[0].values

array([list(['Janusz', 'Pladimir Vutin', 'Eva Pladimirova']),
       list(['Joe Ding']),
       list(['Janusz', 'Pladimir Vutin', 'Eva Pladimirova']),
       list(['Joe Ding']), list([]), list(['Joe Ding'])], dtype=object)

In [69]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
122438658
0 match
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['US']
PAYLOAD
122438659
0 match
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['Poland', 'PL']


In [74]:
out_payload[[i for i in out_payload.columns if "hit" in i and "agg" in i]]

,ap_all_hit_types_aggregated,wl_all_hit_types_aggregated,all_hit_type_aggregated
0,[Janusz],[Janusz],"[{'name': ['PARTY1_NAME_FULL'], 'country': ['A..."
1,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."


In [71]:
import pickle
with open("tests/shared/parsed_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [72]:
import pickle
with open("tests/shared/parsed_payload.pkl", "rb") as f:
    p = pickle.load(f)

In [21]:
p

[{'alertedParty': {'headerInfo': {'currentVersionId': '122438658',
    'datasetId': '1044',
    'datasetName': 'R_US_Active_Party',
    'firstVersionCreatedDt': '2020-01-08T09:31:30.264-05:00',
    'inputVersionSample': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.',
    'lastVersionUpdatedDt': '2020-01-08T09:31:30.264-05:00',
    'masterId': '72951854',
    'masterVersion': '412740c151535f0756e8dbec7440b726c7a3e135',
    'stopDescriptors': {'stopDescriptor': {'name': 'Joe Doe'}},
    'uniqueCustomerId': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'},
   'inputRecordHist': {'inputRecords': [{'createdDate': '01/08/20',
      'fields': [{'isScreenable': 'false',
        'name': 'SOURCE_REF',
        'sortOrder': '1',
        'value': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'},
       {'isScreenable': 'false',
        'name': 'UNIQUE_KEY',
        'sortOrder': '2',
        'value': 'A05003324172'},
       {'isScreenable': 'false', 'name': 'BARCODE

In [31]:
with open(f'notebooks/sample/wm_address_in_payload_format_2_input_3_match_records.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [32]:
payload = payload_json

In [33]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [34]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

2022-04-13 10:41:43,170 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_country. Ignore for HIT TYPE AGENT
2022-04-13 10:41:43,171 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_job. Ignore for HIT TYPE AGENT
2022-04-13 10:41:43,172 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_other. Ignore for HIT TYPE AGENT
2022-04-13 10:41:43,173 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_country. Ignore for HIT TYPE AGENT
2022-04-13 10:41:43,173 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_job. Ignore for HIT TYPE AGENT
2022-04-13 10:41:43,174 — main.engine — WARNING — merge_to_target_col_from_source_col

In [35]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_company_names_aggregated,wl_all_company_names_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,...,wl_all_hit_types_aggregated,ap_all_matched_tokens_tp_markeds_aggregated,wl_all_matched_tokens_tp_markeds_aggregated,ap_all_ap_name_tp_markeds_aggregated,wl_all_ap_name_tp_markeds_aggregated,ap_all_ap_id_tp_markeds_aggregated,wl_all_ap_id_tp_markeds_aggregated,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated,all_hit_type_aggregated
0,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]","[022368917, 154421273]",[],"[Pladimir Vutin, Eva Pladimirova]",[Joe Ding],"[Pladimir Vutin, Eva Pladimirova]",[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[I],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
1,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]","[022368917, 154421273]",[],"[Pladimir Vutin, Eva Pladimirova]",[Joe Ding],"[Pladimir Vutin, Eva Pladimirova]",[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[I],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
2,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]","[022368917, 154421273]",[],"[Pladimir Vutin, Eva Pladimirova]",[Joe Dong],"[Pladimir Vutin, Eva Pladimirova]",[Joe Dong],[],[Joe Dong],...,[],[],[],[],[],[],[],[I],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."


In [36]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
123
0 match
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['US']
PAYLOAD
122438659
0 match
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['US']
PAYLOAD
122438659
0 match
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['Poland', 'PL']


In [37]:
import pickle
with open("tests/shared/parsed_payload_2_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [38]:
new_dict = {}
def return_key(dict_, prefix):
    
    if isinstance(dict_, list):
        for num, i in enumerate(dict_):
            return_key(i, prefix +f"[{num}]")
        return
    if isinstance(dict_, str) or dict_ is None:
        new_dict[prefix] = dict_
        return

    for key in dict_:
        if prefix:
            basic_prefix = prefix + "." + key
        else:
            basic_prefix = key
        return_key(dict_[key], basic_prefix)

In [39]:
with open(f'notebooks/sample/wm_party_payload_without_supplemental_info.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [21]:
payload = payload_json

In [22]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [23]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

2022-04-13 10:36:00,285 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_country. Ignore for HIT TYPE AGENT
2022-04-13 10:36:00,285 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_job. Ignore for HIT TYPE AGENT
2022-04-13 10:36:00,286 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_other. Ignore for HIT TYPE AGENT
2022-04-13 10:36:00,287 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_country. Ignore for HIT TYPE AGENT
2022-04-13 10:36:00,287 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_job. Ignore for HIT TYPE AGENT
2022-04-13 10:36:00,288 — main.engine — WARNING — merge_to_target_col_from_source_col

In [24]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_company_names_aggregated,wl_all_company_names_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,...,wl_all_hit_types_aggregated,ap_all_matched_tokens_tp_markeds_aggregated,wl_all_matched_tokens_tp_markeds_aggregated,ap_all_ap_name_tp_markeds_aggregated,wl_all_ap_name_tp_markeds_aggregated,ap_all_ap_id_tp_markeds_aggregated,wl_all_ap_id_tp_markeds_aggregated,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated,all_hit_type_aggregated
0,[],"[MAY 6, 1981]",[],[],[],[Joe Ding],[],[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[C],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
1,[],"[MAY 6, 1981]",[],[],[],[Joe Dong],[],[Joe Dong],[],[Joe Dong],...,[],[],[],[],[],[],[],[C],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."


In [25]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
122438658
0 match
[] ['MAY 6, 1981']
[] []
PAYLOAD
122438659
0 match
[] ['MAY 6, 1981']
[] ['Poland', 'PL']


In [26]:
import pickle
with open("tests/shared/empty_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [27]:
from etl_pipeline.config import load_agent_config

In [28]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_company_names_aggregated,wl_all_company_names_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,...,wl_all_hit_types_aggregated,ap_all_matched_tokens_tp_markeds_aggregated,wl_all_matched_tokens_tp_markeds_aggregated,ap_all_ap_name_tp_markeds_aggregated,wl_all_ap_name_tp_markeds_aggregated,ap_all_ap_id_tp_markeds_aggregated,wl_all_ap_id_tp_markeds_aggregated,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated,all_hit_type_aggregated
0,[],"[MAY 6, 1981]",[],[],[],[Joe Ding],[],[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[C],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
1,[],"[MAY 6, 1981]",[],[],[],[Joe Dong],[],[Joe Dong],[],[Joe Dong],...,[],[],[],[],[],[],[],[C],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."


In [30]:
from omegaconf import OmegaConf
alert_agents_config = {
        "alert_type": OmegaConf.load(os.path.join(CONFIG_APP_DIR, "agents", "agents.yaml"))
    }


import omegaconf
def get_fields(dict_):
    fields = []
    for key in dict_:
        
        if isinstance(dict_[key], omegaconf.dictconfig.DictConfig):
            new_fields = get_fields(dict_[key])
            fields.extend(new_fields)
        else:
#             import pdb; pdb.set_trace()
            fields.extend(dict_[key])
    return fields

fields = [i.split('.')[-1] for i in get_fields(alert_agents_config['alert_type']) if "alertedParty.inputRecordHist.inputRecords.INPUT_FIELD" not in i]
    
    

from glob import glob

files = glob("pipelines/**/*.py",  recursive=True)
files.extend(glob("etl_pipeline/**/*.py",  recursive=True))

fields_to_check = list(fields)
for file in files:
    with open(file) as f:
        text = f.read()
    new_fields_to_check = []
    
    for field in fields_to_check:
        if field in text:
            continue
        new_fields_to_check.append(field)
    fields_to_check = new_fields_to_check
        
    

fields_to_check

CONFIG_APP_DIR = os.environ["CONFIG_APP_DIR"] 

NameError: name 'CONFIG_APP_DIR' is not defined